About Dataset

Context This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content 5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset. Columns

* asan - ID of the product, like B000FA64PK
* helpful - helpfulness rating of the review - example: 2/3.
* overall - rating of the product.
* reviewText - text of the review (heading).
* reviewTime - time of the review (raw).
* reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
* reviewerName - name of the reviewer.
* summary - summary of the review (description).
* unixReviewTime - unix timestamp.


In [132]:
# Load the dataset
import pandas as pd
data=pd.read_csv('all_kindle_review.csv')
data= data.iloc[:8000,:]


In [133]:
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [134]:
data= data[['reviewText','rating']]
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [135]:
data.shape

(8000, 2)

In [136]:
data.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [137]:
data.duplicated().sum()

0

In [138]:
data['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [139]:
data['rating'].value_counts()

rating
4    2012
5    2003
3    1344
1    1330
2    1311
Name: count, dtype: int64

In [140]:
## postive review is 1 and negative review is 0
data['rating']=data['rating'].apply(lambda x:0 if x<3 else 1)

In [141]:
data['rating']

0       1
1       1
2       1
3       1
4       1
       ..
7995    1
7996    0
7997    0
7998    1
7999    1
Name: rating, Length: 8000, dtype: int64

In [142]:
data['rating'].value_counts()

rating
1    5359
0    2641
Name: count, dtype: int64

Converting to lower cases

In [143]:
data['reviewText']= data['reviewText'].str.lower()

In [144]:
data.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [145]:
import re
import nltk
from nltk.corpus import stopwords

from bs4 import BeautifulSoup
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [146]:
pip install lxml

Removing special characters

In [147]:

data['reviewText']=data['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))

Remove the stopswords

In [148]:
sw_list = stopwords.words('english')
data['reviewText'] = data['reviewText'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

Remove url 

In [149]:
data['reviewText']=data['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))

Remove html tags

In [150]:
data['reviewText'] = data['reviewText'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())


Remove any additional spaces

In [151]:

data['reviewText']=data['reviewText'].apply(lambda x: " ".join(x.split()))

In [152]:
data.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


Lemmatization

In [153]:
from nltk.stem import WordNetLemmatizer

In [154]:
lemmatizer=WordNetLemmatizer()

In [155]:

def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [156]:

data['reviewText']=data['reviewText'].apply(lambda x:lemmatize_words(x))

In [157]:
data.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


Train Test Split

In [158]:

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data['reviewText'],data['rating'],
                                              test_size=0.20)

In [159]:
from scipy.sparse import csr_matrix

In [160]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=100)  # Limit the number of features
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = vectorizer.transform(X_test).toarray()

In [161]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [162]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow=GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

ValueError: Found input variables with inconsistent numbers of samples: [800, 6400]

In [98]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [99]:

y_pred_bow=nb_model_bow.predict(X_test_bow)

In [101]:
y_pred_tfidf=nb_model_bow.predict(X_test_tfidf)

ValueError: X has 100 features, but GaussianNB is expecting 8328 features as input.

In [ ]:
confusion_matrix(y_test,y_pred_bow)

In [ ]:

print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

In [ ]:
confusion_matrix(y_test,y_pred_tfidf)

In [ ]:

print("TFIDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))